In [4]:
import requests

api_url = "https://rest.uniprot.org/uniprotkb/search"

snap25_accession = "P60880"
data = {"query": snap25_accession, "fields": ["id", "gene_names", "organism_name"]}

# headers = 

response = requests.get(api_url, json=data)
response

<Response [400]>

# References

- [Uniprot API: Programmatic access - Retrieving entries via queries](https://www.uniprot.org/help/api_queries)
- [Programmatic access - Retrieving individual entries
](https://www.uniprot.org/help/api_retrieve_entries)
- **[REST API - Access the UniProt website programmatically](https://www.uniprot.org/help/api)**
- **[REST API - Retrieve entries](https://www.uniprot.org/help/api_retrieve_entries)**
- **[REST API - ID Mapping](https://www.uniprot.org/help/id_mapping)**
- **[REST API - Retrieving entries via queries](https://www.uniprot.org/help/api_queries)**
- **[REST API - Downloading](https://www.uniprot.org/help/api_downloading)**